In [1]:
# Jade Chongsathapornpong, June 2023
import os

import fabio
from nexusformat.nexus import *
import pyFAI
import pyFAI.gui
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

%matplotlib nbagg

# Load Geometry Reference
Here we'll load a PONI file generated by pyFAI calibration, e.g. from xrd-cal.ipynb. This will be used to initialize the azimuthal integrator.

In [2]:
poni_file = r"../../Data/ROCK/XRD/lab6_250mm_2.poni" # Modify this

azim_int = pyFAI.load(poni_file)
detector_imsize = azim_int.detector.MAX_SHAPE
print(azim_int)
print("Image Max Size=", detector_imsize)
# print(azim_int.__dict__)

Detector Rayonix SX165	 PixelSize= 3.950e-05, 3.950e-05 m
Wavelength= 1.127129e-10 m
SampleDetDist= 2.616153e-01 m	PONI= 2.022990e-01, 2.348106e-01 m	rot1=-0.026380  rot2=0.002005  rot3=0.000000 rad
DirectBeamDist= 261.707 mm	Center: x=6119.331, y=5134.776 pix	Tilt= 1.516° tiltPlanRotation= 4.347° 𝛌= 1.127Å
Image Max Size= (4096, 4096)


# Read Input Files
Here we choose a directory containing the images that we want to integrate using the loaded geometry reference.

In [3]:
in_dir = r"../../Data/ROCK/XRD/WAM39B" # Modify this
out_dir = r"../../Data/ROCK/XRD/WAM39B_integrated" # Modify this

###

def read_nxs_image(path, imsize=detector_imsize): # Modify if necessary
    """Args: 
        - path: to an *.nxs file with image
        - imsize: tuple, (size_x, size_y)
    Returns:
        - np.ndarray with image of shape (size_x, size_y)"""
    f = nxload(path)
    arr = np.array(f.entry.scan_data.nxsignal).transpose((1,2,0))
    im = Image.fromarray(arr.squeeze()).resize(imsize)
    return np.array(im)
    
def read_edf_image(path, imsize=detector_imsize): # NOT TESTED! May need fixing.
    """See read_nxs_image"""
    arr = fabio.open(path).data
    im = Image.fromarray(arr.squeeze()).resize(imsize)
    return np.array(im)

###

# Look for NeXUS (*.nxs) or European Data Format (*.edf) files and read them in
### Warning: This may take a significant amount of memory (a few GB) but could in principle be modified for lazy evaluation
in_filenames = [] # names of the files in input directory
images = [] # array of images as numpy arrays
out_filepaths = [] # paths to write integrated files to
for item in os.listdir(in_dir):
    
    in_filenames.append(item)
    out_filepaths.append(os.path.join(out_dir, os.path.splitext(item)[0] + '_integrated.dat'))
    
    path = os.path.join(in_dir, item)
    if item.lower().endswith('.nxs'):    
        images.append(read_nxs_image(path))
    elif item.lower().endswith('.edf'):
        images.append(read_edf_image(path))
    else:
        print("Unrecognized extension:", item)

# Integration

In [10]:
for in_filename, image, out_filepath in zip(in_filenames, images, out_filepaths):
    print("Using image from:", in_filename)
    print("Writing to:", out_filepath)
    spectrum = azim_int.integrate1d(image, 1000, unit="2th_deg", filename=out_filepath)

Using image from: no_holder_xrd_000001.nxs
Writing to: ROCK\\WAM39B_integrated\no_holder_xrd_000001_integrated.dat
Using image from: uholder_perp_xrd_000001.nxs
Writing to: ROCK\\WAM39B_integrated\uholder_perp_xrd_000001_integrated.dat
Using image from: uholder_toic_xrd_000001.nxs
Writing to: ROCK\\WAM39B_integrated\uholder_toic_xrd_000001_integrated.dat
Using image from: wam39b_fin_palier_1_xrd_000001.nxs
Writing to: ROCK\\WAM39B_integrated\wam39b_fin_palier_1_xrd_000001_integrated.dat
Using image from: wam39b_fin_palier_6_xrd_000001.nxs
Writing to: ROCK\\WAM39B_integrated\wam39b_fin_palier_6_xrd_000001_integrated.dat
Using image from: wam39b_fin_palier_zn_3_xrd_000001.nxs
Writing to: ROCK\\WAM39B_integrated\wam39b_fin_palier_zn_3_xrd_000001_integrated.dat
Using image from: wam39b_fin_palier_zn_4_xrd_000001.nxs
Writing to: ROCK\\WAM39B_integrated\wam39b_fin_palier_zn_4_xrd_000001_integrated.dat
Using image from: wam39b_rampe_1_xrd_000001.nxs
Writing to: ROCK\\WAM39B_integrated\wam39b_

Using image from: wam39b_rampe_3_xrd_000002.nxs
Writing to: ROCK\\WAM39B_integrated\wam39b_rampe_3_xrd_000002_integrated.dat
Using image from: wam39b_rampe_3_xrd_000003.nxs
Writing to: ROCK\\WAM39B_integrated\wam39b_rampe_3_xrd_000003_integrated.dat
Using image from: wam39b_rampe_3_xrd_000004.nxs
Writing to: ROCK\\WAM39B_integrated\wam39b_rampe_3_xrd_000004_integrated.dat
Using image from: wam39b_rampe_3_xrd_000005.nxs
Writing to: ROCK\\WAM39B_integrated\wam39b_rampe_3_xrd_000005_integrated.dat
Using image from: wam39b_rampe_3_xrd_000006.nxs
Writing to: ROCK\\WAM39B_integrated\wam39b_rampe_3_xrd_000006_integrated.dat
Using image from: wam39b_rampe_3_xrd_000007.nxs
Writing to: ROCK\\WAM39B_integrated\wam39b_rampe_3_xrd_000007_integrated.dat
Using image from: wam39b_rampe_3_xrd_000008.nxs
Writing to: ROCK\\WAM39B_integrated\wam39b_rampe_3_xrd_000008_integrated.dat
Using image from: wam39b_rampe_3_xrd_000009.nxs
Writing to: ROCK\\WAM39B_integrated\wam39b_rampe_3_xrd_000009_integrated.dat
